<a href="https://colab.research.google.com/github/Bao3333/MachineLearning/blob/main/Naive_bayes_poetry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/tibaml0826/raw/main/poem_test.csv"
urlretrieve(url, "test.csv")
url = "https://github.com/Elwing-Chou/tibaml0826/raw/main/poem_train.csv"
urlretrieve(url, "train.csv")

('train.csv', <http.client.HTTPMessage at 0x7f82c6cdf3d0>)

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
test_df = pd.read_csv("test.csv", encoding="utf-8")


In [ ]:
"""
凡事皆物件: 所有東西都是具有型態(Type)
字典: dic  操作: [key]
函式你也要把它分開
函式型態: print           操作: (3+2)
    : 還沒打開的錦囊(步驟集合)  操作: 打開執行
"""

In [ ]:
# example1
b = int
b(4.2)

4

In [ ]:
# example2
def test(n):
    if n == 0:
        return int
    else:
        return round
test(1)(4.24223, 3)

4.242

In [ ]:
# example3-1
l = [-3, 2, 1, 4]
sorted(l, key=abs)

[4, -3, 2, 1]

In [ ]:
# example3-2
students = [
    (170, 80),
    (180, 85),
    (180, 70)
]
def trans(s):
    return s[0]
# 身高小到大, 體重維持原本排序
sorted(students, key=lambda s:s[0])
# 平常的sorted
sorted(students, key=lambda s:(s[0], s[1]))
# 身高小到大, 體重大到小
sorted(students, key=lambda s:(s[0], -s[1]))

In [ ]:
import jieba
def poemcut(p):
  return " ".join(jieba.cut(p))

# series的apply是把函式套到每個指定的series裡
x_train = train_df["內容"].apply(poemcut)
x_test = test_df["內容"].apply(poemcut)

In [ ]:
# 處理答案 -> 帶到skiearn中的都要是數字
# Series.unique()/Series.value_counts()
train_df["作者"].value_counts() # <- 計算不同值得各個數量
poets = train_df["作者"].unique() # <- 計算總共有幾個不同的值

poets2idx = {p:i for i, p in enumerate(poets)} # 語法可見下方示範
idx2poets = {i:p for i, p in enumerate(poets)}

#                            ↓以dict中的key以value取代
y_train = train_df["作者"].replace(poets2idx)
y_test = test_df["作者"].replace(poets2idx)
y_test

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    2
21    2
22    2
23    2
24    2
25    2
26    2
27    2
28    2
29    2
Name: 作者, dtype: int64

In [ ]:
# 語法示範1
ans = []
for i in range(3):
    ans.append(i ** 2)

ans2 = [i ** 2 for i in range(3)]
ans2

[0, 1, 4]


[0, 1, 4]

In [ ]:
# 語法示範2
poets = ["李白", "杜甫", "白居易"]
ans = {}
# enumerate可產生list中的資料以及其對應的索引
for i, p in enumerate(poets):
  ans[p] = i
ans

ans2 = {p:i for i, p in enumerate(poets)}
print(ans)
print(ans2)

# 回上上格

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()

# 利用x_train的資料去訓練有哪些詞(x_test也是用這些詞彙不另外訓練)
vec.fit(x_train)
vec.vocabulary_

In [ ]:
# 再把訓練好的vec拿來統計x_train和x_test出現的詞數量
x_train_count = vec.transform(x_train)
x_test_count = vec.transform(x_test)

# 呈現的形式會是一個Sparse metric稀疏矩陣,此舉陣型是會把值是0的欄位給忽略,以免檔案過大
x_train_count

<2731x52294 sparse matrix of type '<class 'numpy.int64'>'
	with 85677 stored elements in Compressed Sparse Row format>

In [ ]:
# check一下 。 \r\n 等標點符號有沒有被算進去
# vec.vocabulary_["。"] 

In [ ]:
from sklearn.naive_bayes import MultinomialNB
#                     ↓此係數用來作機率的平滑化smoothing,避免0%、100%的出現,透過調整保底的大小來修正準確率
clf = MultinomialNB(alpha=0.25)
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=0.25)

In [ ]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(y_test, pre)

0.8666666666666667

In [ ]:
p = input("請輸入一首詩：")
print(poemcut(p))
pcount = vec.transform([poemcut(p)])
probs = clf.predict_proba(pcount)[0]
print(probs)
for result in zip (poets, probs):
  print(result)

請輸入一首詩：城上歸時晚，庭前宿處危。月明無葉樹，霜滑有風枝。 啼澀飢喉咽，飛低凍翅垂。畫堂鸚鵡鳥，冷暖不相知。
城上 歸時 晚 ， 庭前 宿處 危 。 月 明無葉樹 ， 霜 滑 有 風枝 。   啼澀 飢喉 咽 ， 飛低 凍 翅 垂 。 畫堂 鸚鵡 鳥 ， 冷暖 不 相知 。
[0.80917568 0.02115679 0.16966753]
('李白', 0.8091756805152162)
('杜甫', 0.021156792692373923)
('白居易', 0.16966752679240757)
